# Assignment 3 - Syntax Directed Definition

In this assignment, you are given a grammar `mygrammar/Expr.g4`.  The grammar is a calculator that supports symbol bindings and string operations.

1. Make sure you understand the grammar.
2. Use the provided Makefile to compile the grammar and generate all the necessary ANTLR classes.
3. You must implement the SDD using ANTLR's visitor pattern by completing the two work units in this worksheet.

Load the dependencies.

In [7]:
@file:DependsOn("/data/shared/antlr-4.9.1-complete.jar")
@file:DependsOn(".")

Import the classes

In [8]:
import org.antlr.v4.runtime.*
import mygrammar.*

## Work Unit 1

Declare one or more classes to encapsulate data representation.

You must have a data class that supports the storage and `toString` rendering of data values.

In [9]:
enum class DataType {
    NUMBER,
    STRING,
    NONE
}

class Data() {
    var number: Double = 0.0
    var string: String = ""
    var type: DataType = DataType.NONE
    
    constructor(value: Double): this() {
        this.number = value
        this.type = DataType.NUMBER
    }
    
    constructor(value: String): this() {
        this.string = value
        this.type = DataType.STRING
    }
    
    override fun toString(): String {
        return when(this.type) {
            DataType.NUMBER -> this.number.toString()
            DataType.STRING -> "\"${this.string}\""
            DataType.NONE -> "None"
        }
    }
}

In [10]:
Data()

None

In [11]:
Data(1.0)

1.0

In [12]:
Data("Hello World")

"Hello World"

## Work Unit 2

In this work unit, you are to implement the entire `Visitor` class
by extending the `ExprBaseVisitor<Data>` class.

In [22]:
class Visitor: ExprBaseVisitor<Data>() {
    val scope = mutableMapOf<String, Data>()
    override fun visitProgram(ctx: ExprParser.ProgramContext): Data {
        var x: Data = Data()
        ctx.statement().forEach {
            x = this.visit(it)
        }
        
        return x
    }
    
    override fun visitAssignmentStatement(ctx: ExprParser.AssignmentStatementContext): Data {
        val x = this.visit(ctx.assignment())
        return x
    }
    
    override fun visitExprStatement(ctx: ExprParser.ExprStatementContext): Data 
    = this.visit(ctx.expr())

    override fun visitAssignment(ctx: ExprParser.AssignmentContext): Data {
        val x = this.visit(ctx.expr())
        scope[ctx.ID().text] = x
        return x
    }
    
    override fun visitAddExpr(ctx: ExprParser.AddExprContext): Data {
        val x = this.visit(ctx.x)
        val y = this.visit(ctx.y)
        return when {
            x.type == DataType.NUMBER && y.type == DataType.NUMBER -> Data(x.number + y.number)
            x.type == DataType.STRING && y.type == DataType.STRING -> Data(x.string + y.string)
            x.type == DataType.NUMBER && y.type == DataType.STRING -> Data(x.number.toString() + y.string)
            x.type == DataType.STRING && y.type == DataType.NUMBER -> Data(x.string + y.number.toString())
            else -> run {
                println("Error: addExpr type error")
                Data()
            }
        }
    }
    
    override fun visitSubExpr(ctx: ExprParser.SubExprContext): Data {
        val x = this.visit(ctx.x)
        val y = this.visit(ctx.y)
        return when {
            x.type == DataType.NUMBER && y.type == DataType.NUMBER -> Data(x.number - y.number)
            else -> run {
                println("Error: subExpr type error")
                Data()
            }
        }
    }
    
    override fun visitMulExpr(ctx: ExprParser.MulExprContext): Data {
        val x = this.visit(ctx.x)
        val y = this.visit(ctx.y)
        return when {
            x.type == DataType.NUMBER && y.type == DataType.NUMBER -> Data(x.number * y.number)
            x.type == DataType.STRING && y.type == DataType.NUMBER 
                -> Data(x.string.repeat(y.number.toInt()))
            else -> run {
                println("Error: mulExpr type error")
                Data()
            }
        }
    }
    
    override fun visitDivExpr(ctx: ExprParser.DivExprContext): Data {
        val x = this.visit(ctx.x)
        val y = this.visit(ctx.y)
        return when {
            x.type == DataType.NUMBER && y.type == DataType.NUMBER -> run {
                if(y.number == 0.0) {
                    println("Error: divExpr by zero")
                    Data()
                } else {
                    Data(x.number / y.number)                    
                }
            }
            else -> run {
                println("Error: divExpr type error")
                Data()
            }
        }
    }
    
    override fun visitParenExpr(ctx: ExprParser.ParenExprContext): Data 
    = this.visit(ctx.expr())
    
    override fun visitValueExpr(ctx: ExprParser.ValueExprContext): Data 
    = this.visit(ctx.value())
        
    override fun visitNumericValue(ctx: ExprParser.NumericValueContext): Data
    = Data(ctx.NUMERIC().text.toDouble())
    
    override fun visitStringValue(ctx: ExprParser.StringValueContext): Data 
    = Data(ctx.STRING().text.trim('"'))
    
    override fun visitIdValue(ctx: ExprParser.IdValueContext): Data 
    = this.scope.getOrDefault(ctx.ID().text, Data(0.0))
}

This function is provided to you to run the source code as provided.

In [23]:
fun run(source: String) {
    val input = CharStreams.fromString(source)
    val lexer = ExprLexer(input)
    val tokens = CommonTokenStream(lexer)
    val parser = ExprParser(tokens)
    
    val tree = parser.program()
    val data = Visitor().visit(tree)
    
    println("Program terminated with: " + data)
}

In [24]:
run("1")

Program terminated with: 1.0


In [25]:
run("1+2")

Program terminated with: 3.0


In [26]:
run("1/2")

Program terminated with: 0.5


In [27]:
run("""
"hello"
""")

Program terminated with: "hello"


In [28]:
run("""
"hello" + 2
""")

Program terminated with: "hello2.0"


In [29]:
run("""
"hello" * 2
""")

Program terminated with: "hellohello"


In [30]:
run("""
"hello" + "world"
""")

Program terminated with: "helloworld"


In [31]:
run("""
"hello" + " " + "world"
""")

Program terminated with: "hello world"


In [32]:
run("""
let x = 122
""")

Program terminated with: 122.0


In [33]:
run("""
let x = "Hello"
""")

Program terminated with: "Hello"


In [34]:
run("""
let pi = 3.14
let radius = 2.5
let area = pi * radius * radius

"area is " + area
""")

Program terminated with: "area is 19.625"


In [36]:
run("""
let x = 123
let y = "(x)"
x + y
""")

Program terminated with: "123.0(x)"


In [38]:
run("""
let x = 123
let y = "(x)"
(y+x+",")*4
""")

Program terminated with: "(x)123.0,(x)123.0,(x)123.0,(x)123.0,"
